In [1]:
import getpass
print("Enter you Hugging Face token:")
TOKEN = getpass.getpass()

Enter you Hugging Face token:
··········


In [2]:
!pip install git+https://www.github.com/huggingface/transformers

!pip install git+https://github.com/huggingface/accelerate

!pip install bitsandbytes

!pip install einops
!pip install datasets
!pip install trl
!pip install peft
!pip install -U "huggingface_hub[cli]"

  Cloning https://www.github.com/huggingface/transformers to /tmp/pip-req-build-39m7tor8
  Running command git clone --filter=blob:none --quiet https://www.github.com/huggingface/transformers /tmp/pip-req-build-39m7tor8
  Resolved https://www.github.com/huggingface/transformers to commit fc8764c9a618add64c33e83720f974750bcd0978
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-9tfn_gv3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-9tfn_gv3
  Resolved https://github.com/huggingface/accelerate to commit 4b6be8991059f39a8df8893333d11c54bc51fc60
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!git config --global credential.helper store
!huggingface-cli login --token $TOKEN --add-to-git-credential

Token is valid (permission: write).
The token `colab` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `colab`


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [5]:
# Suppressing “INFO” and “WARNING” messages by setting the verbosity of the Transformers library.
from transformers import logging
logging.set_verbosity_error()

# Suppressing Python warnings
import warnings
warnings.filterwarnings("ignore")

In [6]:
if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
else:
  compute_dtype = torch.float16

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

In [7]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
quantized_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config = bnb_config)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
input = tokenizer("Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?", return_tensors="pt").to('cuda')

response = quantized_model.generate(**input, max_new_tokens = 100)
print(tokenizer.batch_decode(response, skip_special_tokens=True))

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

['Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\xa0\n## Step 1: Determine the number of clips Natalia sold in April.\nNatalia sold 48 clips in April.\n\n## Step 2: Calculate the number of clips Natalia sold in May.\nShe sold half as many clips in May as in April. Half of 48 is 48 / 2 = 24.\n\n## Step 3: Calculate the total number of clips Natalia sold in April and May.\nAdd the number of clips sold in April to the number of']


In [9]:
from datasets import load_dataset

dataset = "openai/gsm8k"
data = load_dataset(dataset, 'main')

tokenizer.pad_token = tokenizer.eos_token
data = data.map(lambda samples: tokenizer(samples["question"], samples["answer"], truncation=True, padding="max_length", max_length=100), batched=True)
train_samples = data["train"].select(range(400))

display(train_samples)

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 400
})

In [10]:
print(train_samples[:1])

{'question': ['Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'], 'answer': ['Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'], 'input_ids': [[128000, 45, 4306, 689, 6216, 27203, 311, 220, 2166, 315, 1077, 4885, 304, 5936, 11, 323, 1243, 1364, 6216, 4376, 439, 1690, 27203, 304, 3297, 13, 2650, 1690, 27203, 1550, 42701, 689, 4662, 31155, 304, 5936, 323, 3297, 30, 128000, 45, 4306, 689, 6216, 220, 2166, 14, 17, 284, 1134, 2166, 14, 17, 28, 1187, 2511, 1187, 27203, 304, 3297, 627, 45, 4306, 689, 6216, 220, 2166, 10, 1187, 284, 1134, 2166, 10, 1187, 28, 5332, 2511, 5332, 27203, 31155, 304, 5936, 323, 3297, 627, 827, 220, 5332, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [11]:
import peft
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

In [20]:
from transformers import TrainingArguments
import os

working_dir = 'content/drive/MyDrive/large-models/'
output_directory = os.path.join(working_dir, "lora")

training_args = TrainingArguments(
    output_dir = output_directory,
    auto_find_batch_size = True,
    learning_rate = 3e-4,
    num_train_epochs=2,
    report_to = "none"
)

In [21]:
import transformers
from trl import SFTTrainer

trainer = SFTTrainer(
    model = quantized_model,
    args = training_args,
    train_dataset = train_samples,
    peft_config = lora_config, tokenizer = tokenizer,
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

In [22]:
trainer.train()

{'train_runtime': 1274.1604, 'train_samples_per_second': 0.628, 'train_steps_per_second': 0.157, 'train_loss': 1.3610220336914063, 'epoch': 2.0}


TrainOutput(global_step=200, training_loss=1.3610220336914063, metrics={'train_runtime': 1274.1604, 'train_samples_per_second': 0.628, 'train_steps_per_second': 0.157, 'train_loss': 1.3610220336914063, 'epoch': 2.0})

In [23]:
# Save the model.
model_path = os.path.join(output_directory, f"lora_model")

trainer.model.save_pretrained(model_path)

In [7]:
#We are going to clean some variables to avoid memory problems
import gc
import torch
del quantized_model
del trainer
del train_samples
del data
torch.cuda.empty_cache()
gc.collect()

NameError: name 'quantized_model' is not defined

In [8]:
model_path = "content/drive/MyDrive/large-models/lora/lora_model"

from peft import AutoPeftModelForCausalLM
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

loaded_model = AutoPeftModelForCausalLM.from_pretrained(
                                        model_path,
                                        quantization_config = bnb_config)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
input = tokenizer("Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?", return_tensors="pt").to('cuda')

response = loaded_model.generate(**input, max_new_tokens = 100)
print(tokenizer.batch_decode(response, skip_special_tokens=True))

['Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?Question 1: How many clips did Natalia sell in May?\n48 / 2 = <<48/2=24>>24\nNatalia sold 24 clips in May.\nQuestion 2: How many clips did Natalia sell in April and May altogether?\n48 + 24 = <<48+24=72>>72\nNatalia sold 72 clips in April and May altogether.\n#### 72 ####\n#### 72 ####\n#### 72 ####\n#### ']
